In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local[10]","block")

In [2]:
import time

In [3]:
input_rdd = sc.textFile("./pagerank_data.txt")

In [4]:
BLOCK_SIZE = [16,8,4,2,1]
MAX_ITER_COUNT = 3
HYPER_PARAMETER = 0.85
time_result = []
CONVERGENCE_TOL = 0.0001

In [5]:
count = 0 

In [6]:
data = []

f = open('./pagerank_data.txt', 'r')
while True:
    line = f.readline()
    if not line:
        break
    temp = line.split(' ')
    data.append((int(temp[0]), [int(i) for i in temp[1:]]))

print(len(data))


30000


In [7]:
data[1]

(1,
 [1,
  2,
  3,
  12290,
  5,
  6,
  7,
  8,
  4,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  12302,
  18,
  19,
  20,
  21,
  22,
  23,
  17,
  25,
  26,
  27,
  28,
  29,
  30,
  24,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  4134,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  9,
  53,
  55,
  56,
  57,
  58,
  54,
  60,
  61,
  62,
  63,
  64,
  10305,
  66,
  67,
  68,
  69,
  10307,
  71,
  72,
  73,
  74,
  18507,
  76,
  77,
  78,
  4166,
  80,
  81,
  82,
  2131,
  84,
  85,
  86,
  83,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  87,
  96,
  2144,
  98,
  97,
  100,
  101,
  102,
  103,
  104,
  10345,
  106,
  107,
  108,
  109,
  105,
  111,
  14448,
  112,
  114,
  2163,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  4216,
  2166,
  124,
  127,
  2167,
  129,
  130,
  131,
  128,
  133,
  134,
  135,
  132,
  4233,
  137,
  139,
  2181,
  141,
  142,
  138,
  144,
  2191,
  146,
  147,
  143,
  149,
  150,
  151,
  152,
  153,

In [8]:
input_rdd = input_rdd.map(lambda x : [int(y) for y in x.split(' ')]).map(lambda x : (x[0],x[1:]))

In [9]:
data1 = []

f = open('./inverted_list.txt', 'r')
while True:
    line = f.readline()
    if not line:
        break
    temp = line.split(' ')

    if temp[-1] == '':
        data1.append((int(temp[0]),[]))
    else:
        data1.append((int(temp[0]), [int(i) for i in temp[1:]]))

print(len(data1))


30000


In [11]:
time_result = []
each_size = sc.broadcast([len(d[1]) for d in data])
for BLOCK in BLOCK_SIZE:
	
	EACH_BLOCK_SIZE = len(data) // BLOCK
	test = [sc.parallelize(data1[b*EACH_BLOCK_SIZE:(b+1)*EACH_BLOCK_SIZE]) for b in range(BLOCK)]

	time_start = time.time()
	for trial in range(5):
		
		p_i = sc.broadcast([1/30000 for _ in range(30000)])
		print(f'BLOCK SIZE : {BLOCK}, {trial}번째 시작')
		for j in range(MAX_ITER_COUNT):
			temp_p_i = [(1-HYPER_PARAMETER)/30000 for _ in range(30000)]
			iter_time_start = time.time()
			err = 0
			for t in test:
				key_pair = t.flatMap(lambda x:[ (x[0],p_i.value[i] / each_size.value[x[0]]) for i in x[1]])
				new_p_i = key_pair.reduceByKey(lambda x,y : x+y)
				post_process_p_i = new_p_i.map(lambda x : (x[0],HYPER_PARAMETER*x[1] + (1-HYPER_PARAMETER)/30000))
				result = post_process_p_i.collect()

				for r in result:
					temp_p_i[r[0]] = r[1]

			err += sum([abs(p_i.value[i] - temp_p_i[i]) for i in range(30000)])

			if err < CONVERGENCE_TOL:
				break
			print("======================================================================================================================")
			print(f'BLOCK SIZE : {BLOCK}, {trial}번째, {j+1}th iteration')
			print(f'takes {time.time()-iter_time_start}seconds, ERR : {err}')
			print(temp_p_i[:5])
			print("======================================================================================================================")

			p_i = sc.broadcast(temp_p_i)

	time_result.append([(time.time()-time_start)/5])

BLOCK SIZE : 16, 0번째 시작
BLOCK SIZE : 16, 0번째, 1th iteration
takes 25.728102922439575seconds, ERR : 1.0219486458462346
[5e-06, 0.0008541508491511088, 0.0008541508491511088, 0.000853303393213556, 0.000853303393213556]
BLOCK SIZE : 16, 0번째, 2th iteration
takes 24.44144606590271seconds, ERR : 0.037013493829941575
[5e-06, 0.0008541541519203593, 0.0008541541519203593, 0.0008533066926869475, 0.0008533066926869475]
BLOCK SIZE : 16, 0번째, 3th iteration
takes 26.86532425880432seconds, ERR : 0.0008227230154214779
[5e-06, 0.0008541542462547997, 0.0008541542462547997, 0.0008533067869272048, 0.0008533067869272048]
BLOCK SIZE : 16, 1번째 시작
BLOCK SIZE : 16, 1번째, 1th iteration
takes 39.61476421356201seconds, ERR : 1.0219486458462346
[5e-06, 0.0008541508491511088, 0.0008541508491511088, 0.000853303393213556, 0.000853303393213556]
BLOCK SIZE : 16, 1번째, 2th iteration
takes 35.84878993034363seconds, ERR : 0.037013493829941575
[5e-06, 0.0008541541519203593, 0.0008541541519203593, 0.0008533066926869475, 0.0008

KeyboardInterrupt: 

In [ ]:
time_start = time.time()
p_i = sc.broadcast([1/30000 for _ in range(30000)])
each_size = sc.broadcast([len(d[1]) for d in data])
BLOCK = 16
err = 0
EACH_BLOCK_SIZE = len(data) // BLOCK

test = [sc.parallelize(data1[b*EACH_BLOCK_SIZE:(b+1)*EACH_BLOCK_SIZE]) for b in range(BLOCK)]

for j in range(MAX_ITER_COUNT):
  temp_p_i = [(1-HYPER_PARAMETER)/30000 for _ in range(30000)]
  
  for t in test:
    key_pair = t.flatMap(lambda x:[ (x[0],p_i.value[i] / each_size.value[x[0]]) for i in x[1]])
    new_p_i = key_pair.reduceByKey(lambda x,y : x+y)
    post_process_p_i = new_p_i.map(lambda x : (x[0],HYPER_PARAMETER*x[1] + (1-HYPER_PARAMETER)/30000))
    result = post_process_p_i.collect()

    for r in result:
      temp_p_i[r[0]] = r[1]

  err += sum([abs(p_i.value[i] - temp_p_i[i]) for i in range(30000)])

  if err < CONVERGENCE_TOL:
    break
  print(temp_p_i[:5])
  # print(err,temp_p_i)
  p_i = sc.broadcast(temp_p_i)

time_result.append(time.time()-time_start)

[5e-06, 0.0008541508491511088, 0.0008541508491511088, 0.000853303393213556, 0.000853303393213556]
[5e-06, 0.0008541541519203593, 0.0008541541519203593, 0.0008533066926869475, 0.0008533066926869475]
[5e-06, 0.0008541542462547997, 0.0008541542462547997, 0.0008533067869272048, 0.0008533067869272048]
[5e-06, 0.0008541542460313729, 0.0008541542460313729, 0.0008533067867039916, 0.0008533067867039916]
[5e-06, 0.0008541542461332184, 0.0008541542461332184, 0.0008533067868057218, 0.0008533067868057218]
[5e-06, 0.0008541542461323242, 0.0008541542461323242, 0.0008533067868048521, 0.0008533067868048521]
[5e-06, 0.000854154246132377, 0.000854154246132377, 0.0008533067868048834, 0.0008533067868048834]
[5e-06, 0.0008541542461323589, 0.0008541542461323589, 0.0008533067868048839, 0.0008533067868048839]
[5e-06, 0.0008541542461323663, 0.0008541542461323663, 0.0008533067868048831, 0.0008533067868048831]
[5e-06, 0.0008541542461323626, 0.0008541542461323626, 0.0008533067868048797, 0.0008533067868048797]
[5e-

In [ ]:
time_result

NameError: name 'time_result' is not defined